In [119]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [120]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [121]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [122]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [123]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h", 
                    "price_change_percentage_7d","price_change_percentage_14d", "price_change_percentage_30d",
                    "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"]])

# Review the scaled data
market_scaled [0:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [124]:
# Create a DataFrame with the scaled data
market_scaled_df = pd.DataFrame(market_scaled, columns=["price_change_percentage_24h", 
                    "price_change_percentage_7d","price_change_percentage_14d", "price_change_percentage_30d",
                    "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"])

# Copy the crypto names from the original data (fisrt create a new column called 'coin_id' and add the data from original df's index)
market_scaled_df["coin_id"] = df_market_data.index

# Set the coinid column as index for market_scaled_df
market_scaled_df = market_scaled_df.set_index("coin_id")

# Display sample data
market_scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [125]:
# Create a list with the number of k-values from 1 to 11

k_values = list(range(1, 11))

# Display the list
print(k_values)



[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [126]:


# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia for each k value
for i in k_values: 
# Loop through k values from 1 to 11
    # 1. Create a KMeans model with i clusters
    k_model = KMeans(n_clusters=i, random_state=0)
    
    # 2. Fit the model to the scaled data
    k_model.fit(df_market_scaled)
    
    # 3. Append the inertia value to the list
    inertia.append(k_model.inertia_)

#  Display the inertia values
print(inertia)




[287.0, 198.5718180979813, 145.63574772192862, 79.02243535120975, 66.21778006000338, 59.13907002537009, 48.40631449041361, 38.99358798927969, 34.40059545400234, 28.57360256096785]


In [127]:


# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k_original": k_values,  # Assuming 'k' is a list of k values from 1 to 11
              "inertia": inertia}  # 'inertia' is the list of inertia values you calculated

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_original = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow_original.head()



,k_original,inertia
0,1,287.000000
1,2,198.571818
2,3,145.635748
3,4,79.022435
4,5,66.217780


In [128]:
# Plot a line chart with all the inertia values computed with the different values of k
hvplot.extension('bokeh') 
elbow_plot = df_elbow_original.hvplot.line(
    x="k_original",  # k values on the x-axis
    y="inertia",     # Inertia values on the y-axis
    title="Elbow Curve 1",  # Title of the plot
    xticks=k  # This will make sure the x-axis ticks correspond to the k values
)

elbow_plot


:Curve   [k_original]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [129]:

# Initialise the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [130]:
# Fit the K-Means model using the scaled data
model.fit(df_market_scaled)

KMeans(n_clusters=4, random_state=0)

In [131]:
# Predict the clusters to group the cryptocurrencies using the scaled data
kmeans_predictions = model.predict(df_market_scaled)

# Print the resulting array of cluster values.
kmeans_predictions

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0],
      dtype=int32)

In [132]:
# Create a copy of the DataFrame
df_market_scaled_prediction=df_market_scaled.copy()

In [133]:
print(type(df_market_scaled_prediction))


<class 'numpy.ndarray'>


In [134]:
# Define the column names (based on the given data)
column_names = [
    "price_change_percentage_24h", "price_change_percentage_7d",
    "price_change_percentage_14d", "price_change_percentage_30d",
    "price_change_percentage_60d", "price_change_percentage_200d",
    "price_change_percentage_1y"
]

# Convert the NumPy array to a Pandas DataFrame with column names
df_market_scaled_prediction = pd.DataFrame(df_market_scaled_prediction, columns=column_names)

# Add the 'Prediction' column
df_market_scaled_prediction['Prediction'] = kmeans_predictions

# Display the updated DataFrame
print(df_market_scaled_prediction.head())


   price_change_percentage_24h  price_change_percentage_7d  \
0                     0.508529                    0.493193   
1                     0.185446                    0.934445   
2                     0.021774                   -0.706337   
3                    -0.040764                   -0.810928   
4                     1.193036                    2.000959   

   price_change_percentage_14d  price_change_percentage_30d  \
0                     0.772200                     0.235460   
1                     0.558692                    -0.054341   
2                    -0.021680                    -0.061030   
3                     0.249458                    -0.050388   
4                     1.760610                     0.545842   

   price_change_percentage_60d  price_change_percentage_200d  \
0                    -0.067495                     -0.355953   
1                    -0.273483                     -0.115759   
2                     0.008005                     -0.55

In [135]:

# Display sample data
df_market_scaled_prediction.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Prediction
0,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
1,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
2,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
3,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
4,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [136]:
import hvplot.pandas  # Ensure you have hvPlot installed

# Create a scatter plot with hvPlot
scatter_plot = df_market_scaled_prediction.hvplot.scatter(
    x="price_change_percentage_24h",                # X-axis variable
    y="price_change_percentage_7d",                 # Y-axis variable
    c="Prediction",                                 # Color by the 'Prediction' column
    hover_cols=["coin_id"],                         # Add coin_id to hover information
    title="Cryptocurrency Clusters (K-Means)",      # Add a title
    xlabel="24h Price Change (%)",                  # X-axis label
    ylabel="7d Price Change (%)",                   # Y-axis label
    height=400,                                     # Height of the plot
    width=600,                                      # Width of the plot
    cmap="Category10"                               # Color map for clusters
)

# Display the plot
scatter_plot


:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,Prediction)

---

### Optimise Clusters with Principal Component Analysis.

In [137]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [138]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.ith your scaled data

# Apply PCA to reduce the data to 3 components
market_data_pca = pca.fit_transform(df_market_scaled_prediction)

# View the first five rows of the PCA-transformed data
print(market_data_pca[:5])


[[ 1.19408229 -0.90207402 -0.58533763]
 [ 1.00970605 -0.66358438 -1.13695032]
 [-0.72253591 -0.30763241  0.7218133 ]
 [-0.74826646 -0.33237895  0.55847869]
 [ 2.25853853 -1.82696614 -1.37816649]]


In [139]:
# Retrieve the explained variance ratio for each principal component
explained_variance = pca.explained_variance_ratio_

# Display the explained variance
print("Explained Variance Ratio for each principal component:", explained_variance)

# Optionally, calculate the cumulative variance to see how much total variance is explained by the first 3 components
cumulative_variance = explained_variance.cumsum()
print("Cumulative Explained Variance:", cumulative_variance)


Explained Variance Ratio for each principal component: [0.37269822 0.32489961 0.18917649]
Cumulative Explained Variance: [0.37269822 0.69759782 0.88677431]


In [140]:
sum(pca.explained_variance_ratio_)

0.8867743104975919

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 88%

In [141]:
# Create a new DataFrame with the PCA data.
market_pca_df = pd.DataFrame(
    market_data_pca,
    columns = ["PCA1", "PCA2", "PCA3"]
)

# Copy the crypto names from the original data
market_pca_df['coin_id'] = df_market_data.index


# Set the coinid column as index
market_pca_df = market_pca_df.set_index("coin_id")

# Display sample data
market_pca_df.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,1.194082,-0.902074,-0.585338
ethereum,1.009706,-0.663584,-1.136950
tether,-0.722536,-0.307632,0.721813
ripple,-0.748266,-0.332379,0.558479
bitcoin-cash,2.258539,-1.826966,-1.378166


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [142]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))
k_values

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [143]:
from sklearn.cluster import KMeans

# Create an empty list to store the inertia values
inertia_values = []

# Loop through the k-values from 1 to 10
for k_value in k_values:
    # Create a KMeans model with the current value of k
    kmeans = KMeans(n_clusters=k_value, random_state=42)
    
    # Fit the KMeans model to the PCA-transformed data
    kmeans.fit(market_data_pca)
    
    # Append the inertia value for the current model
    inertia_values.append(kmeans.inertia_)

# Display the inertia values
print(inertia_values)



[290.0184568161516, 199.10805307785148, 112.40120116244475, 103.88808569697605, 35.06010302455461, 30.45270570091619, 19.649877928230474, 16.316381252463355, 14.786068344455966, 12.958588172813029]


In [144]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia_values}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)
df_elbow.head()

,k,inertia
0,1,290.018457
1,2,199.108053
2,3,112.401201
3,4,103.888086
4,5,35.060103


In [145]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve 2", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** k=4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, the k value is same as the original data

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [146]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame


# Print the resulting array of cluster values.


In [147]:
# Initialise the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)


In [148]:
# Fit the K-Means model using the PCA data
model.fit(market_pca_df)

KMeans(n_clusters=4, random_state=1)

In [149]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters_predition = model.predict(market_pca_df)

# Print the resulting array of cluster values.
print(clusters_predition)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 3 1 0 0 2
 0 0 0 0]


In [150]:
# Create a copy of the DataFrame with the PCA data
clusters_predition_df= market_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
clusters_predition_df['clusters_predition'] = clusters_predition

# Display sample data
clusters_predition_df.head()

,PCA1,PCA2,PCA3,clusters_predition
coin_id,,,,
bitcoin,1.194082,-0.902074,-0.585338,1
ethereum,1.009706,-0.663584,-1.136950,1
tether,-0.722536,-0.307632,0.721813,0
ripple,-0.748266,-0.332379,0.558479,0
bitcoin-cash,2.258539,-1.826966,-1.378166,1


In [151]:
# Create a scatter plot using hvPlot by setting 
clusters_predition_df.hvplot.scatter(
    
# x="PC1"` and `y="PC2"
    x="PCA1",
    y="PCA2",
    
# Color the graph points with the labels found using K-Means and 
    by = 'clusters_predition',
     title="PCA Scatter Plot of Cryptocurrencies",
    
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
     hover_cols='coin_id'
)

:NdOverlay   [clusters_predition]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualise and Compare the Results

In this section, you will visually analyse the cluster analysis results by contrasting the outcome with and without using the optimisation techniques.

In [152]:
import pandas as pd

# Define the first dataframe (df_elbow) with inertia values for curve 1
df_elbow = pd.DataFrame({
    'k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'inertia': [287.0, 198.5718180979813, 145.63574772192862, 
                79.02243535120975, 66.21778006000338, 59.13907002537009, 
                48.40631449041361, 38.99358798927969, 34.40059545400234, 
                28.57360256096785]
})

# Define the second dataframe (df_elbow2) with inertia values for curve 2
df_elbow2 = pd.DataFrame({
    'k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'inertia': [290.0184568161516, 199.10805307785148, 112.40120116244475, 
                103.88808569697605, 35.06010302455461, 30.45270570091619, 
                19.649877928230474, 16.316381252463355, 14.786068344455966, 
                12.958588172813029]
})

# Plot the first elbow curve (df_elbow)
elbow_curve_1 = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve 1", 
    xticks=k
)

# Plot the second elbow curve (df_elbow2)
elbow_curve_2 = df_elbow2.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve 2", 
    xticks=k
)



# Composite plot to contrast the Elbow curves
df_elbow.hvplot.line(x="k", y="inertia", title= "Elbow Curve 1", xticks=k) + df_elbow2.hvplot.line(x="k", y="inertia", title= "Elbow Curve 2",xticks=k)


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [153]:


# Scatter plot 1: Based on percentage changes
scatter_plot = df_market_scaled_prediction.hvplot.scatter(
    x="price_change_percentage_24h",                # X-axis variable
    y="price_change_percentage_7d",                 # Y-axis variable
    c="Prediction",                                 # Color by the 'Prediction' column
    hover_cols=["coin_id"],                         # Add coin_id to hover information
    title="Cryptocurrency Clusters (K-Means)",      # Add a title
    xlabel="24h Price Change (%)",                  # X-axis label
    ylabel="7d Price Change (%)",                   # Y-axis label
    height=400,                                     # Height of the plot
    width=600,                                      # Width of the plot
    cmap="Category10"                               # Color map for clusters
)

# Scatter plot 2: Based on PCA components
pca_plot = clusters_predition_df.hvplot.scatter(
    x="PCA1",                                       # X-axis variable for PCA
    y="PCA2",                                       # Y-axis variable for PCA
    by="clusters_predition",                        # Color by the 'clusters_predition' column
    title="PCA Scatter Plot of Cryptocurrencies",   # Add a title
    hover_cols=["coin_id"],                         # Add coin_id to hover information
    xlabel="PCA Component 1",                       # X-axis label
    ylabel="PCA Component 2",                       # Y-axis label
    height=400,                                     # Height of the plot
    width=600,                                      # Width of the plot
    cmap="Category10"                               # Color map for clusters
)

# Combine the two plots into a composite plot
composite_plot = scatter_plot + pca_plot

# Display the composite plot
composite_plot


:Layout
   .Scatter.I   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,Prediction)
   .NdOverlay.I :NdOverlay   [clusters_predition]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analysing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Based on these observations, it seems that using fewer features (PCA data) to cluster the cryptocurrency data using K-Means has had a positive impact. It has resulted in better-defined and separable clusters compared to using the original data. The reduction in the number of features through PCA has likely helped to highlight the essential patterns and reduce the impact of noise, leading to more accurate and meaningful clustering results.